In [1]:
from stellargraph import StellarGraph
import pandas as pd
import multiprocessing
import numpy as np
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
import pickle
from stellargraph.data import UniformRandomMetaPathWalk
from gensim.models import Word2Vec
warnings.filterwarnings("ignore")

2025-02-22 14:58:52.385285: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 14:58:52.387185: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 14:58:52.427298: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-22 14:58:52.428200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-22 14:58:53.277962: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

# Final code from here

In [2]:
#user set params
dir_path = "WSJ split data files"
dbname = "WSJ"
epochs = 100

In [ ]:
all_nodes = train_df["src"].append(train_df["dest"]).unique()
len(all_nodes)

In [ ]:
all_nodes = pd.Series(list(all_nodes))
nodes_entity = all_nodes[all_nodes.between(5000, 99999)].unique()
nodes_users = all_nodes[all_nodes>=100000].unique()
nodes_events = all_nodes[all_nodes<5000].unique()
nodes_entity = pd.DataFrame(index=list(nodes_entity))
nodes_users = pd.DataFrame(index=list(nodes_users))
nodes_events = pd.DataFrame(index=list(nodes_events))
pickle.dump(nodes_entity,open("nodes_entity.pickle","wb"))
pickle.dump(nodes_events,open("nodes_events.pickle","wb"))
pickle.dump(nodes_users,open("nodes_users.pickle","wb"))

In [ ]:
#other params 
dimensions = 64
num_walks = 1
walk_length = 100
context_window_size = 10
num_iter = 1
workers = multiprocessing.cpu_count()
user_metapaths = [
    #entity metapaths
    ["entity","entity"],
    ["entity", "events", "entity"],
    ["entity", "events", "entity", "entity"],
    ["entity", "events", "events", "entity"],
    ["entity", "user", "entity"],
    ["entity", "user", "entity", "entity"],
    ["entity", "user", "user", "entity"],
    ["entity", "user", "events", "entity"],
    ["entity", "events", "user", "entity"],
    ["entity", "user", "events", "entity", "entity"],
    ["entity", "events", "user", "entity", "entity"],
    #events metapath
    ["events", "events"],
    ["events", "entity", "events"],
    ["events", "entity", "events", "events"],
    ["events", "entity", "entity", "events"],
    ["events", "user", "events"],
    ["events", "user", "events", "events"],
    ["events", "user", "user", "events"],
    ["events", "entity", "user", "events"],
    ["events", "user", "entity", "events"],
    ["events", "entity", "user", "events", "events"],
    ["events", "user", "entity", "events", "events"],
    #user metapath
    ["user", "user"],
    ["user", "events", "user"],
    ["user", "events", "user", "user"],
    ["user", "events", "events", "user"],
    ["user", "entity", "user"],
    ["user", "entity", "user", "user"],
    ["user", "entity", "entity", "user"],
    ["user", "entity", "events", "user"],
    ["user", "events", "entity", "user"],
    ["user", "entity", "events", "user", "user"],
    ["user", "events", "entity", "user", "user"],
]

In [17]:
for split_no in range(1,11,1):
    test_file_no = split_no
    print("\n\n\n Test file no : ",test_file_no)
    
    #creating training and testing dataset
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            del df
        else:
            test_df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv")
            neg_df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
            test_df = test_df.append(neg_df, ignore_index=True)
            del neg_df
    #converting to StellarGraph
    train_df.rename(columns = {'src':'source','dest':'target'}, inplace = True)
    G = StellarGraph({"entity": nodes_entity, "user": nodes_users, "events": nodes_events},edges=train_df)
    print("Graph ready")
    
    #training walks
    curr = time.time()
    rw = UniformRandomMetaPathWalk(G)
    walks = rw.run(
        G.nodes(), n=num_walks, length=walk_length, metapaths=user_metapaths
    )
    print(f"Number of random walks for 'Train Graph': {len(walks)}")

    timetaken = time.time()-curr
    print("time taken for training walks : ",timetaken)
    
    #training word2vec
    curr = time.time()
    mdl = Word2Vec(
            walks,
            vector_size=dimensions,
            window=context_window_size,
            min_count=0,
            sg=1,
            workers=workers
        )
    timetaken = time.time()-curr
    print("time taken for training word2vec : ",timetaken)
    mdl.save("my_"+str(dbname)+"_metapath2V_node_embedding_model_"+str(test_file_no)+".model")
    
    #calculating node features
    curr = time.time()
    node_ids = mdl.wv.index_to_key  # list of node IDs
    node_embeddings = (
        mdl.wv.vectors
    )
    timetaken = time.time()-curr
    print("time taken for getting node embeddings : ",timetaken)
    
    #creating training data
    train_df = pd.DataFrame()
    for i in range(1,11,1):
        if i != test_file_no:
            df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
            train_df = train_df.append(df, ignore_index=True)
            df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
            train_df = train_df.append(df, ignore_index=True)
            del df
    train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
    train_df = train_df.sample(frac=1).reset_index(drop=True)
    train_df.loc[train_df['label'] == -1, "label"] = 0
    # creating x_train
    X_train = [(mdl.wv.get_vector(node_ids.index(train_df['src'][i])) + mdl.wv.get_vector(node_ids.index(train_df['dest'][i]))) for i in train_df.index]
    X_train = np.array(X_train)
    y_train = np.array(train_df['label'])
    del train_df
    
    #preparing x_test data
    # creating x_test
    #test_df = test_df.sample(frac=1).reset_index(drop=True)
    #test_df.loc[test_df['label'] == -1, "label"] = 0
    #X_test = [(mdl.wv.get_vector(node_ids.index(test_df['src'][i])) + mdl.wv.get_vector(node_ids.index(test_df['dest'][i]))) for i in test_df.index]
    #pickle.dump(X_test,open("metapath2v_"+str(dbname)+"_X_test_whole_"+str(test_file_no)+".csv","wb"))
    #X_test = np.array(X_test)
    #y_test = list(test_df['label'])
    del test_df
    
    print("x_train and x_test are ready")
    # creating keras model
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
    model.add(Dense(1, activation='sigmoid'))

    curr = time.time()

    # compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X_train, y_train, validation_split=0.10, epochs=epochs, batch_size=16,verbose=1)

    timetaken = time.time()-curr
    print("time taken for training : ",timetaken)
    print("time taken for per epoch : ",timetaken/epochs)
    
    model.save("my_"+str(dbname)+"_METAPATH2V_model_"+str(test_file_no)+".keras")




 Test file no :  1
Graph ready
Number of random walks for 'Train Graph': 93896
time taken for training walks :  859.9173460006714
time taken for training word2vec :  19.719265699386597
time taken for getting node embeddings :  3.814697265625e-06
x_train and x_test are ready
Epoch 1/100
64092/64092 [==============================] - 182s 3ms/step - loss: 0.4050 - accuracy: 0.8286 - val_loss: 0.3852 - val_accuracy: 0.8365
Epoch 2/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3764 - accuracy: 0.8429 - val_loss: 0.3721 - val_accuracy: 0.8459
Epoch 3/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3696 - accuracy: 0.8457 - val_loss: 0.3693 - val_accuracy: 0.8456
Epoch 4/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3659 - accuracy: 0.8479 - val_loss: 0.3655 - val_accuracy: 0.8490
Epoch 5/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3639 - accuracy: 0.8487 - val_loss: 0.3665 

Epoch 54/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3501 - accuracy: 0.8552 - val_loss: 0.3590 - val_accuracy: 0.8511
Epoch 55/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3500 - accuracy: 0.8551 - val_loss: 0.3554 - val_accuracy: 0.8527
Epoch 56/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3499 - accuracy: 0.8553 - val_loss: 0.3589 - val_accuracy: 0.8522
Epoch 57/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3500 - accuracy: 0.8553 - val_loss: 0.3567 - val_accuracy: 0.8530
Epoch 58/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3500 - accuracy: 0.8553 - val_loss: 0.3531 - val_accuracy: 0.8537
Epoch 59/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3499 - accuracy: 0.8552 - val_loss: 0.3558 - val_accuracy: 0.8511
Epoch 60/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3499 - accuracy:

Epoch 61/100
64092/64092 [==============================] - 180s 3ms/step - loss: 0.3591 - accuracy: 0.8502 - val_loss: 0.3630 - val_accuracy: 0.8480
Epoch 62/100
64092/64092 [==============================] - 179s 3ms/step - loss: 0.3593 - accuracy: 0.8500 - val_loss: 0.3619 - val_accuracy: 0.8490
Epoch 63/100
64092/64092 [==============================] - 178s 3ms/step - loss: 0.3592 - accuracy: 0.8500 - val_loss: 0.3646 - val_accuracy: 0.8489
Epoch 64/100
64092/64092 [==============================] - 179s 3ms/step - loss: 0.3591 - accuracy: 0.8498 - val_loss: 0.3642 - val_accuracy: 0.8483
Epoch 65/100
64092/64092 [==============================] - 179s 3ms/step - loss: 0.3591 - accuracy: 0.8498 - val_loss: 0.3627 - val_accuracy: 0.8481
Epoch 66/100
64092/64092 [==============================] - 181s 3ms/step - loss: 0.3591 - accuracy: 0.8501 - val_loss: 0.3647 - val_accuracy: 0.8465
Epoch 67/100
64092/64092 [==============================] - 181s 3ms/step - loss: 0.3592 - accuracy:

Epoch 68/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3590 - accuracy: 0.8501 - val_loss: 0.3612 - val_accuracy: 0.8493
Epoch 69/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3590 - accuracy: 0.8502 - val_loss: 0.3611 - val_accuracy: 0.8485
Epoch 70/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3590 - accuracy: 0.8502 - val_loss: 0.3629 - val_accuracy: 0.8472
Epoch 71/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3589 - accuracy: 0.8503 - val_loss: 0.3632 - val_accuracy: 0.8487
Epoch 72/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3589 - accuracy: 0.8503 - val_loss: 0.3626 - val_accuracy: 0.8486
Epoch 73/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3590 - accuracy: 0.8504 - val_loss: 0.3618 - val_accuracy: 0.8491
Epoch 74/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3589 - accuracy:

Epoch 75/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3581 - accuracy: 0.8508 - val_loss: 0.3625 - val_accuracy: 0.8486
Epoch 76/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3580 - accuracy: 0.8510 - val_loss: 0.3637 - val_accuracy: 0.8469
Epoch 77/100
64092/64092 [==============================] - 183s 3ms/step - loss: 0.3580 - accuracy: 0.8510 - val_loss: 0.3654 - val_accuracy: 0.8482
Epoch 78/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3580 - accuracy: 0.8508 - val_loss: 0.3646 - val_accuracy: 0.8479
Epoch 79/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3579 - accuracy: 0.8510 - val_loss: 0.3672 - val_accuracy: 0.8462
Epoch 80/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3579 - accuracy: 0.8510 - val_loss: 0.3727 - val_accuracy: 0.8441
Epoch 81/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3580 - accuracy:

...

Epoch 89/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3487 - accuracy: 0.8561 - val_loss: 0.3501 - val_accuracy: 0.8554
Epoch 90/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3487 - accuracy: 0.8563 - val_loss: 0.3525 - val_accuracy: 0.8554
Epoch 91/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3486 - accuracy: 0.8561 - val_loss: 0.3497 - val_accuracy: 0.8565
Epoch 92/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3486 - accuracy: 0.8562 - val_loss: 0.3489 - val_accuracy: 0.8557
Epoch 93/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3485 - accuracy: 0.8564 - val_loss: 0.3497 - val_accuracy: 0.8555
Epoch 94/100
64092/64092 [==============================] - 184s 3ms/step - loss: 0.3486 - accuracy: 0.8561 - val_loss: 0.3487 - val_accuracy: 0.8558
Epoch 95/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3485 - accuracy:

Epoch 96/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3537 - accuracy: 0.8544 - val_loss: 0.3579 - val_accuracy: 0.8532
Epoch 97/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3536 - accuracy: 0.8544 - val_loss: 0.3569 - val_accuracy: 0.8522
Epoch 98/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3536 - accuracy: 0.8543 - val_loss: 0.3567 - val_accuracy: 0.8523
Epoch 99/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3537 - accuracy: 0.8544 - val_loss: 0.3572 - val_accuracy: 0.8518
Epoch 100/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3537 - accuracy: 0.8543 - val_loss: 0.3602 - val_accuracy: 0.8494
time taken for training :  19217.01907634735
time taken for per epoch :  192.1701907634735



 Test file no :  8
Graph ready
Number of random walks for 'Train Graph': 93896
time taken for training walks :  983.4895722866058
time taken for training w

x_train and x_test are ready
Epoch 1/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.4180 - accuracy: 0.8202 - val_loss: 0.4014 - val_accuracy: 0.8297
Epoch 2/100
64092/64092 [==============================] - 185s 3ms/step - loss: 0.3944 - accuracy: 0.8319 - val_loss: 0.3931 - val_accuracy: 0.8314
Epoch 3/100
64092/64092 [==============================] - 186s 3ms/step - loss: 0.3890 - accuracy: 0.8349 - val_loss: 0.3899 - val_accuracy: 0.8354
Epoch 4/100
64092/64092 [==============================] - 186s 3ms/step - loss: 0.3860 - accuracy: 0.8364 - val_loss: 0.3867 - val_accuracy: 0.8352
Epoch 5/100
64092/64092 [==============================] - 186s 3ms/step - loss: 0.3828 - accuracy: 0.8378 - val_loss: 0.3834 - val_accuracy: 0.8378
Epoch 6/100
64092/64092 [==============================] - 186s 3ms/step - loss: 0.3799 - accuracy: 0.8395 - val_loss: 0.3798 - val_accuracy: 0.8394
Epoch 7/100
64092/64092 [==============================] - 185s 3ms/step - lo

64092/64092 [==============================] - 189s 3ms/step - loss: 0.3627 - accuracy: 0.8507 - val_loss: 0.3637 - val_accuracy: 0.8494
Epoch 8/100
64092/64092 [==============================] - 188s 3ms/step - loss: 0.3617 - accuracy: 0.8509 - val_loss: 0.3611 - val_accuracy: 0.8504
Epoch 9/100
64092/64092 [==============================] - 188s 3ms/step - loss: 0.3609 - accuracy: 0.8510 - val_loss: 0.3655 - val_accuracy: 0.8484
Epoch 10/100
64092/64092 [==============================] - 189s 3ms/step - loss: 0.3603 - accuracy: 0.8515 - val_loss: 0.3616 - val_accuracy: 0.8493
Epoch 11/100
64092/64092 [==============================] - 189s 3ms/step - loss: 0.3597 - accuracy: 0.8515 - val_loss: 0.3601 - val_accuracy: 0.8510
Epoch 12/100
64092/64092 [==============================] - 188s 3ms/step - loss: 0.3591 - accuracy: 0.8517 - val_loss: 0.3596 - val_accuracy: 0.8499
Epoch 13/100
64092/64092 [==============================] - 188s 3ms/step - loss: 0.3587 - accuracy: 0.8521 - val_l

Epoch 62/100
64092/64092 [==============================] - 214s 3ms/step - loss: 0.3535 - accuracy: 0.8545 - val_loss: 0.3557 - val_accuracy: 0.8529
Epoch 63/100
64092/64092 [==============================] - 215s 3ms/step - loss: 0.3534 - accuracy: 0.8543 - val_loss: 0.3567 - val_accuracy: 0.8527
Epoch 64/100
64092/64092 [==============================] - 215s 3ms/step - loss: 0.3534 - accuracy: 0.8543 - val_loss: 0.3592 - val_accuracy: 0.8519
Epoch 65/100
64092/64092 [==============================] - 215s 3ms/step - loss: 0.3534 - accuracy: 0.8544 - val_loss: 0.3563 - val_accuracy: 0.8519
Epoch 66/100
64092/64092 [==============================] - 214s 3ms/step - loss: 0.3534 - accuracy: 0.8545 - val_loss: 0.3558 - val_accuracy: 0.8527
Epoch 67/100
64092/64092 [==============================] - 214s 3ms/step - loss: 0.3533 - accuracy: 0.8546 - val_loss: 0.3569 - val_accuracy: 0.8521
Epoch 68/100
64092/64092 [==============================] - 214s 3ms/step - loss: 0.3533 - accuracy:

# rough code here

In [3]:
dir_path = "WSJ split data files"
test_file_no = 1

In [4]:
train_df = pd.DataFrame()
for i in range(1,11,1):
    if i != test_file_no:
        df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
        train_df = train_df.append(df, ignore_index=True)
        del df

<ipython-input-4-562bfdb627f2>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-562bfdb627f2>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-562bfdb627f2>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-562bfdb627f2>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-4-562bfdb627f2>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a 

In [5]:
train_df

,src,dest,layer,weight,sign,label
0,5046,8395,12,2,-2.0,1
1,100036,103501,3,27,-2.0,1
2,100670,102593,3,2,-2.0,1
3,870,5764,2,33,-1.0,1
4,5036,6048,12,7,-2.0,1
...,...,...,...,...,...,...
569695,100442,100451,3,4,-2.0,1
569696,5338,8134,12,3,-2.0,1
569697,100075,100086,3,5,-2.0,1
569698,5082,5400,12,2,-2.0,1


In [8]:
s1 = pd.Series([1, 2, 3])
s2 = pd.Series([4, 5, 6])
s1.append(s2)

<ipython-input-8-2f4674388914>:3: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s1.append(s2)


0    1
1    2
2    3
0    4
1    5
2    6
dtype: int64

In [11]:
all_nodes = train_df["src"].append(train_df["dest"]).unique()
len(all_nodes)

<ipython-input-11-4952a3502677>:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_nodes = train_df["src"].append(train_df["dest"]).unique()


8536

In [13]:
all_nodes = pd.Series(list(all_nodes))
#all_nodes[all_nodes.between(5000, 100000)].unique()

array([5046, 5036, 5143, ..., 8168, 7908, 7930])

In [25]:
nodes_entity = all_nodes[all_nodes.between(5000, 99999)].unique()
nodes_users = all_nodes[all_nodes>=100000].unique()
nodes_events = all_nodes[all_nodes<5000].unique()

In [26]:
nodes_entity = pd.DataFrame(index=list(nodes_entity))
nodes_users = pd.DataFrame(index=list(nodes_users))
nodes_events = pd.DataFrame(index=list(nodes_events))
#train_df.rename(columns = {'src':'source','dest':'target'}, inplace = True)

In [28]:

G = StellarGraph({"entity": nodes_entity, "user": nodes_users, "events": nodes_events},edges=train_df)
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 8536, Edges: 569700

 Node types:
  entity: [4857]
    Features: none
    Edge types: entity-default->entity, entity-default->events, entity-default->user
  user: [3651]
    Features: none
    Edge types: user-default->entity, user-default->events, user-default->user
  events: [28]
    Features: none
    Edge types: events-default->entity, events-default->events, events-default->user

 Edge types:
    user-default->user: [422780]
        Weights: range=[1, 2226], mean=12.8169, std=29.887
        Features: float32 vector, length 3
    entity-default->entity: [119500]
        Weights: range=[2, 2973], mean=8.03984, std=36.8979
        Features: float32 vector, length 3
    entity-default->user: [16955]
        Weights: range=[1, 92], mean=2.71525, std=3.50289
        Features: float32 vector, length 3
    entity-default->events: [5570]
        Weights: range=[1, 325], mean=3.8088, std=12.8579
        Features: float32 vector, length 3
    event

In [34]:

dimensions = 128
num_walks = 1
walk_length = 100
context_window_size = 10
num_iter = 1
workers = multiprocessing.cpu_count()
user_metapaths = [
    #entity metapaths
    ["entity","entity"],
    ["entity", "events", "entity"],
    ["entity", "events", "entity", "entity"],
    ["entity", "events", "events", "entity"],
    ["entity", "user", "entity"],
    ["entity", "user", "entity", "entity"],
    ["entity", "user", "user", "entity"],
    ["entity", "user", "events", "entity"],
    ["entity", "events", "user", "entity"],
    ["entity", "user", "events", "entity", "entity"],
    ["entity", "events", "user", "entity", "entity"],
    #events metapath
    ["events", "events"],
    ["events", "entity", "events"],
    ["events", "entity", "events", "events"],
    ["events", "entity", "entity", "events"],
    ["events", "user", "events"],
    ["events", "user", "events", "events"],
    ["events", "user", "user", "events"],
    ["events", "entity", "user", "events"],
    ["events", "user", "entity", "events"],
    ["events", "entity", "user", "events", "events"],
    ["events", "user", "entity", "events", "events"],
    #user metapath
    ["user", "user"],
    ["user", "events", "user"],
    ["user", "events", "user", "user"],
    ["user", "events", "events", "user"],
    ["user", "entity", "user"],
    ["user", "entity", "user", "user"],
    ["user", "entity", "entity", "user"],
    ["user", "entity", "events", "user"],
    ["user", "events", "entity", "user"],
    ["user", "entity", "events", "user", "user"],
    ["user", "events", "entity", "user", "user"],
]

In [35]:
from stellargraph.data import UniformRandomMetaPathWalk
from gensim.models import Word2Vec


def metapath2vec_embedding(graph, name):
    rw = UniformRandomMetaPathWalk(graph)
    walks = rw.run(
        graph.nodes(), n=num_walks, length=walk_length, metapaths=user_metapaths
    )
    print(f"Number of random walks for '{name}': {len(walks)}")

    model = Word2Vec(
        walks,
        size=dimensions,
        window=context_window_size,
        min_count=0,
        sg=1,
        workers=workers,
        iter=num_iter,
    )

    def get_embedding(u):
        return model.wv[u]

    return get_embedding

In [36]:
embedding_train = metapath2vec_embedding(G, "Train Graph")

Number of random walks for 'Train Graph': 93896


TypeError: __init__() got an unexpected keyword argument 'size'

In [38]:
import time
curr = time.time()
rw = UniformRandomMetaPathWalk(G)
walks = rw.run(
    G.nodes(), n=num_walks, length=walk_length, metapaths=user_metapaths
)
print(f"Number of random walks for 'Train Graph': {len(walks)}")

timetaken = time.time()-curr
print("time taken for training walks : ",timetaken)

Number of random walks for 'Train Graph': 93896
time taken for training walks :  1362.9943737983704


In [60]:
import pickle
pickle.dump( walks, open( "walks_m2v_1.p", "wb" ) )

In [41]:
curr = time.time()
model = Word2Vec(
        walks,
        vector_size=dimensions,
        window=context_window_size,
        min_count=0,
        sg=1,
        workers=workers
    )
timetaken = time.time()-curr
print("time taken for training word2vec : ",timetaken)

time taken for training word2vec :  51.37902116775513


In [43]:
def get_embedding(u):
    return model.wv[u]

embedding_train = get_embedding

In [44]:
embedding_train

<function __main__.get_embedding(u)>

In [50]:
curr = time.time()
node_ids = model.wv.index_to_key  # list of node IDs
node_embeddings = (
    model.wv.vectors
)
timetaken = time.time()-curr
print("time taken for getting node embeddings : ",timetaken)

time taken for getting node embeddings :  0.0001201629638671875


In [51]:
node_embeddings

array([[ 0.04454675, -0.15098633, -0.14263374, ..., -0.02475462,
         0.02735988, -0.04422437],
       [ 0.00534413, -0.00093344, -0.07663991, ..., -0.1003122 ,
         0.05472428, -0.02785431],
       [-0.03314362,  0.05169779, -0.16807988, ..., -0.06688873,
         0.11019027, -0.0072902 ],
       ...,
       [-0.04662629, -0.36680228,  0.25478554, ..., -0.05306149,
         0.08224034,  0.16038658],
       [-0.04669413, -0.05721235,  0.15926437, ...,  0.01061422,
        -0.16594188,  0.14179882],
       [-0.00074436, -0.03708893,  0.19749694, ..., -0.00540493,
        -0.05588885,  0.07483114]], dtype=float32)

In [52]:
node_ids

[1114,
 870,
 1100,
 1146,
 5008,
 5009,
 1118,
 5005,
 5011,
 5034,
 5070,
 931,
 5002,
 5042,
 505,
 38,
 1087,
 70,
 5036,
 5012,
 103,
 5014,
 1164,
 1007,
 5255,
 848,
 5057,
 5028,
 5032,
 5145,
 5024,
 5225,
 964,
 5064,
 1163,
 5037,
 5018,
 5000,
 853,
 5053,
 5081,
 5624,
 5095,
 5838,
 5019,
 6,
 5046,
 989,
 5182,
 5264,
 5158,
 5475,
 847,
 120,
 100007,
 5076,
 5045,
 100031,
 100103,
 5494,
 100051,
 900,
 34,
 5223,
 5049,
 100294,
 100170,
 5030,
 5639,
 5385,
 5173,
 5058,
 100519,
 5137,
 5003,
 50,
 5139,
 5688,
 5035,
 5073,
 102786,
 100044,
 921,
 1148,
 100428,
 5051,
 5159,
 100288,
 5366,
 100014,
 5082,
 100747,
 5837,
 100060,
 101524,
 100057,
 101870,
 1180,
 100319,
 5689,
 102231,
 100215,
 100341,
 100216,
 7495,
 5117,
 102391,
 100790,
 5330,
 5060,
 100316,
 100008,
 5525,
 5021,
 5601,
 5536,
 5478,
 5572,
 514,
 5109,
 100036,
 100829,
 100292,
 100252,
 5784,
 5069,
 103045,
 100055,
 5157,
 100405,
 100082,
 100310,
 100070,
 101773,
 5096,
 1014

# finally creating training and testing data

In [54]:
node_ids.index(5005)

7

In [55]:
train_df = pd.DataFrame()
test_file_no = 1
dir_path = "WSJ split data files"
for i in range(1,11,1):
    if i != test_file_no:
        df = pd.read_csv(dir_path+"/WSJ_positive_edges_split_"+str(i)+".csv") 
        train_df = train_df.append(df, ignore_index=True)
        df = pd.read_csv(dir_path+"/WSJ_negative_edges_split_"+str(i)+".csv")
        train_df = train_df.append(df, ignore_index=True)
        del df
train_df = train_df.drop(['layer', 'weight',"sign"], axis=1)
train_df = train_df.sample(frac=1).reset_index(drop=True)

# creating x_train
X_train = [(model.wv.get_vector(node_ids.index(train_df['src'][i])) + model.wv.get_vector(node_ids.index(train_df['dest'][i]))) for i in train_df.index]
X_train = np.array(X_train)
y_train = np.array(train_df['label'])

del train_df

<ipython-input-55-fdc3888e92fb>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-55-fdc3888e92fb>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-55-fdc3888e92fb>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-55-fdc3888e92fb>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_df = train_df.append(df, ignore_index=True)
<ipython-input-55-fdc3888e92fb>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas 

NameError: name 'np' is not defined

In [58]:
temp = model.wv.get_vector(node_ids.index(870))
temp.shape

(128,)

In [57]:
X_train = np.array(X_train)
y_train = np.array(train_df['label'])

del train_df
X_train.shape

(1139400, 128)

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
 
epochs = 100

model = Sequential()
model.add(Dense(56, input_shape=(X_train.shape[1],), activation='relu')) # Add an input shape! (features,)
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

curr = time.time()

# compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X_train, y_train, validation_split=0.10, epochs=epochs, batch_size=64,verbose=2)

timetaken = time.time()-curr
print("time taken for training : ",timetaken)
print("time taken for per epoch : ",timetaken/epochs)

Epoch 1/100
16023/16023 - 18s - loss: -2.2777e+06 - accuracy: 0.1480 - val_loss: -8.0011e+06 - val_accuracy: 0.1491 - 18s/epoch - 1ms/step
Epoch 2/100
16023/16023 - 19s - loss: -2.4414e+07 - accuracy: 0.1487 - val_loss: -4.8366e+07 - val_accuracy: 0.1506 - 19s/epoch - 1ms/step
Epoch 3/100
16023/16023 - 16s - loss: -9.0262e+07 - accuracy: 0.1489 - val_loss: -1.4406e+08 - val_accuracy: 0.1484 - 16s/epoch - 1ms/step
Epoch 4/100
16023/16023 - 17s - loss: -2.2220e+08 - accuracy: 0.1485 - val_loss: -3.1724e+08 - val_accuracy: 0.1501 - 17s/epoch - 1ms/step
Epoch 5/100
16023/16023 - 18s - loss: -4.4242e+08 - accuracy: 0.1487 - val_loss: -5.9013e+08 - val_accuracy: 0.1493 - 18s/epoch - 1ms/step
Epoch 6/100
16023/16023 - 17s - loss: -7.7264e+08 - accuracy: 0.1485 - val_loss: -9.8488e+08 - val_accuracy: 0.1493 - 17s/epoch - 1ms/step
Epoch 7/100
16023/16023 - 17s - loss: -1.2344e+09 - accuracy: 0.1485 - val_loss: -1.5227e+09 - val_accuracy: 0.1495 - 17s/epoch - 1ms/step
Epoch 8/100
16023/16023 - 1

Epoch 60/100
16023/16023 - 18s - loss: -7.7861e+11 - accuracy: 0.1484 - val_loss: -8.0203e+11 - val_accuracy: 0.1494 - 18s/epoch - 1ms/step
Epoch 61/100
16023/16023 - 18s - loss: -8.1808e+11 - accuracy: 0.1486 - val_loss: -8.4231e+11 - val_accuracy: 0.1494 - 18s/epoch - 1ms/step
Epoch 62/100
16023/16023 - 19s - loss: -8.5881e+11 - accuracy: 0.1485 - val_loss: -8.8396e+11 - val_accuracy: 0.1493 - 19s/epoch - 1ms/step
Epoch 63/100
16023/16023 - 17s - loss: -9.0097e+11 - accuracy: 0.1484 - val_loss: -9.2694e+11 - val_accuracy: 0.1494 - 17s/epoch - 1ms/step
Epoch 64/100
16023/16023 - 17s - loss: -9.4438e+11 - accuracy: 0.1487 - val_loss: -9.7129e+11 - val_accuracy: 0.1493 - 17s/epoch - 1ms/step
Epoch 65/100
16023/16023 - 23s - loss: -9.8919e+11 - accuracy: 0.1485 - val_loss: -1.0170e+12 - val_accuracy: 0.1494 - 23s/epoch - 1ms/step
Epoch 66/100
16023/16023 - 16s - loss: -1.0355e+12 - accuracy: 0.1486 - val_loss: -1.0642e+12 - val_accuracy: 0.1493 - 16s/epoch - 1ms/step
Epoch 67/100
16023/1

In [61]:
model.save("my_metapath2vec_model_1.keras")

In [62]:
pickle.dump( X_train, open( "X_train_m2v_1.p", "wb" ) )
pickle.dump( y_train, open( "y_train_m2v_1.p", "wb" ) )

In [45]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding, binary_operator
):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=2000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(
    clf, link_examples_test, link_labels_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [46]:
def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def run_link_prediction(binary_operator):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )

    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_l1, operator_l2]

In [47]:
binary_operators

[<function __main__.operator_l1(u, v)>, <function __main__.operator_l2(u, v)>]

In [48]:

results = [run_link_prediction(op) for op in binary_operators]
best_result = max(results, key=lambda result: result["score"])

print(f"Best result from '{best_result['binary_operator'].__name__}'")

pd.DataFrame(
    [(result["binary_operator"].__name__, result["score"]) for result in results],
    columns=("name", "ROC AUC score"),
).set_index("name")

NameError: name 'examples_train' is not defined